In [49]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


In [50]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

In [51]:
df = pd.read_csv('01. output/aux.csv')

In [52]:
df.drop(columns=['Unnamed: 0', 'text'], inplace=True)

In [53]:
aux = pd.melt(df, id_vars=['Speaker', 'party'])

In [54]:
aux.sort_values(by=['variable'], inplace=True)
aux = aux.reset_index(drop=True)

In [55]:
lox_de_lox = aux[['Speaker', 'party', 'value']].groupby(by=['Speaker', 'party']).sum().reset_index()
lox_de_lox.rename(columns={'value':'m_it'},inplace=True)

In [56]:
aux = pd.merge(
    aux,
    lox_de_lox,
    how='left',
    on=['Speaker', 'party']
)

In [57]:
# On Calcule les termes ds le sigma 
aux['q_hat_it'] = aux['value']/aux['m_it']

In [58]:
aux_Lab = aux.loc[aux['party'] == 'Lab']
aux_Con = aux.loc[aux['party'] == 'Con']

In [59]:
aux_Con['Somme_c_it'] = aux['value'].sum()
aux_Con['Somme_c_it'] = aux_Con['Somme_c_it'] - aux_Con['value']
lox_de_lox = aux_Con[['Speaker', 'm_it']].groupby(by=['Speaker']).sum().reset_index()
lox_de_lox.rename(columns={'m_it':'Somme_m_it'},inplace=True)
aux_Con = pd.merge(
    aux_Con,
    lox_de_lox,
    how='left',
    on=['Speaker']
)
aux_Con['Somme_m_it'] = aux_Con['Somme_m_it'] - aux_Con['value']
aux_Con['q_hat_jt'] = aux_Con['Somme_c_it'] / aux_Con['Somme_m_it']

In [61]:
aux_Lab['Somme_c_it'] = aux['value'].sum()
aux_Lab['Somme_c_it'] = aux_Lab['Somme_c_it'] - aux_Lab['value']
lox_de_lox = aux_Lab[['Speaker', 'm_it']].groupby(by=['Speaker']).sum().reset_index()
lox_de_lox.rename(columns={'m_it':'Somme_m_it'},inplace=True)
aux_Lab = pd.merge(
    aux_Lab,
    lox_de_lox,
    how='left',
    on=['Speaker']
)
aux_Lab['Somme_m_it'] = aux_Lab['Somme_m_it'] - aux_Lab['value']
aux_Lab['q_hat_jt'] = aux_Lab['Somme_c_it'] / aux_Lab['Somme_m_it']